# Hidden Markov Models

## Generating data

In [41]:
import numpy as np
import matplotlib.pyplot as plt

In [42]:
#let's say we have only two hidden states
z_val = np.array([0,1])

prob = np.array([.5,.5])
A = np.array([[.98,.02],[.02,.98]]) #transition matrix
B_pars = np.array([[0,.5],[5,.5]])

In [43]:
def gaussian(x,mu,sigma):
    return (1/(sigma*np.sqrt(2*np.pi)))*np.exp(-((x-mu)**2)/(2*sigma**2))

xt = np.arange(-5,10,.1)
y1=[]
y2=[]
for i in range(len(xt)):
    y1.append(gaussian(xt[i],B_pars[0][0], B_pars[0][1]))
    y2.append(gaussian(xt[i],B_pars[1][0], B_pars[1][1]))
    
plt.plot(xt,y1)
plt.plot(xt,y2)
print(len(xt))

150


In [44]:
def generate_initial_state(N, prob, B_pars):
    z = np.empty((N)) #hidden state sequence
    x = np.empty((N)) #observations
    
    #Generating first state and emission
    r = np.random.random()
    z[0] = np.where(r <= prob[0], 0, 1)
    x[0] = np.random.normal(loc=B_pars[0][0], scale=B_pars[0][1])

    return z, x

def generate_sequence(N, z, x, c, B_pars):
    for i in range(1, N):
        a = np.random.random()
        if z[i-1] == 0:
            z[i] = np.where(a <= c[0], 0, 1)
            x[i] = np.random.normal(loc=B_pars[0][0], scale=B_pars[0][1])
        else:
            z[i] = np.where(a <= c[1], 1, 0)
            x[i] = np.random.normal(loc=B_pars[1][0], scale=B_pars[1][1])
    return z, x


In [45]:
N = 100 #length of sequence
c = np.array([A[0,0],A[1,1]])# Transition probabilities

#generate the arrays z and x for the hidden state sequence and observations
z,x=generate_initial_state(N, prob, B_pars)
z,x=generate_sequence(N, z, x, c, B_pars)

In [46]:
plt.figure(figsize=(8,8))
plt.plot(np.arange(0,N,1),x[:N])

In [47]:
x_0 = x[np.where(z==0)]
print(x_0[:10])

[ 5.74683934 -0.13415026 -0.97854171 -1.16287575 -0.03146137  0.31576852
  0.12475215 -0.98157641 -0.22035203 -0.20558185]


In [48]:
plt.hist(x_0, bins=70, range=(-5,10), density=True)
plt.plot(xt,y1,color='red')

In [49]:
#unique_0, counts_0 = np.unique(x_0, return_counts=True)

In [50]:
#plt.plot(unique_0,counts_0,'o')

In [51]:
x_0.mean()

0.07784918102058348

In [52]:
x_1 = x[np.where(z==1)]

In [53]:
plt.hist(x_1, bins=70, range=(-5,10), density=True)
plt.plot(xt,y2,color='red')

In [54]:
#plt.plot(unique_1,counts_1,'o')

In [55]:
x_1.mean()

4.6752336484668255

## Forward and backward algorithm
The forward and backward algorithm are dynamic programming algorithms that compute the probability of observing a sequence of observations given an HMM. The forward algorithm works by computing the alpha values recursively using the initial probabilities, transition probabilities, and observation probabilities of the HMM. The backward algorithm proceeds in a similar way but it starts from the last time step, working backwards.

The alpha variable represents the probability of observing the first t observations and being in state i at time t:
\begin{equation}
\alpha_t(i) = P(x_{1:t}, z_t = i)
\end{equation}
where $x_{1:t}$ are the first t observations, $z_t$ is the state at time t.

The beta variable represents the probability of observing the remaining observations from time t+1 to the end, given that we are in state i at time t: 
\begin{equation}
\beta_t(i) = P(x_{t+1:T} | z_t = i)
\end{equation}
where $x_{t+1:T}$ are the remaining observations after time t.
The alpha and beta variable are typically computed recursively using the following formulas:

\begin{equation}
\alpha_{t}(j) = \sum_{i=1}^{N} \alpha_{t-1}(i) a_{ij} b_j(z_{t+1})
\end{equation}

\begin{equation}
\beta_t(i) = \sum_{j=1}^{N} a_{ij} b_j(x_{t+1}) \beta_{t+1}(j)
\end{equation}

where $N$ is the number of states in the HMM, $a_{ij}$ is the probability of transitioning from state i to state j, $b_j(x_{t+1})$ is the probability of observing the next observation $x_{t+1}$ given that we are in state j. The $\alpha_t(i)$ term is the alpha value for state i at time t and $\beta_{t+1}(j)$ is the beta value for state j at time $t+1$.

## Gamma and Psi
\begin{equation}
\gamma_t(i) = \frac{\alpha_t(i)\beta_t(i)}{\sum_{j=1}^{N}\alpha_t(j)\beta_t(j)}
\end{equation}

where $N$ is the number of states in the HMM, $\alpha_t(i)$ is computed at time $t$ and state $i$, and $\beta_t(i)$ is computed at time $t$ and state $i$. This formula computes the probability of being in state $i$ at time $t$, given the observed sequence and the HMM parameters.

The formula for the psi value at time step $t$, current state $i$, and next state $j$ is:

\begin{equation}
\psi_t(i,j) = \frac{\alpha_t(i)a_{ij}b_j(x_{t+1})\beta_{t+1}(j)}{\sum_{i=1}^{N}\sum_{j=1}^{N}\alpha_t(i)a_{ij}b_j(x_{t+1})\beta_{t+1}(j)}
\end{equation}

where $N$ is the number of states in the HMM, $a_{ij}$ is the transition probability from state $i$ to state $j$, $b_j(x_{t+1})$ is the emission probability of the observation $x_{t+1}$ given state $j$, and $\beta_{t+1}(j)$ is the backward variable at time $t+1$ and state $j$. 

In [56]:
A = np.array([[.98,.02],[.02,.98]]) #transition matrix

def calculate_emission_matrix(B_pars, x):
    b = np.zeros((2, len(x)))# Create emission probability matrix b
    T=len(x)
    for i in range(T):
        b[0, i] = gaussian(x[i], B_pars[0][0], B_pars[0][1])
        b[1, i] = gaussian(x[i], B_pars[1][0], B_pars[1][1])
    return b


def forward_algorithm(x, A, b, pi=prob[0]):
    # x: observed sequence (array of shape (T,))
    # A: transition probability matrix (array of shape (2, 2))
    # b: emission probability matrix (array of shape (2, T))
    # pi: initial state distribution (array of shape (2,))
    # returns: alpha, the forward variable (array of shape (2, T))

    T = len(x)
    alpha = np.zeros((2, T))

    # initialization of the first alpha
    alpha[:, 0] = pi * b[:, 0]

    # recursion
    for t in range(1, T):
        for j in range(2):
            alpha[j, t] = np.sum(alpha[:, t-1] * A[:, j]) * b[j, t]

    return alpha


def backward_algorithm(x, A, b):
    # x: observed sequence (array of shape (T,))
    # A: transition probability matrix (array of shape (2, 2))
    # b: emission probability matrix (array of shape (2, T))
    # returns: beta, the backward variable (array of shape (2, T))

    T = len(x)
    beta = np.zeros((2, T))

    # initialization of the last beta
    #we assume that in the HMM the probability of observing the last observation given any state at time T is 1
    #Here -1 index represents the last column of the matrix, regardless of the total number of columns
    beta[:, -1] = 1

    # recursion
    for t in range(T-2, -1, -1):
        for j in range(2):
            beta[j, t] = np.sum(beta[:, t+1]*A[j, :]*b[:, t+1])

    return beta

In [57]:
b=calculate_emission_matrix(B_pars, x)
alphas=forward_algorithm(x, A, b, pi=prob[0])
betas =backward_algorithm(x, A, b)

In [58]:
print(z)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]


In [59]:
print(alphas)

[[3.81169531e-02 4.37734600e-15 6.88275538e-31 8.82390212e-27
  5.04985540e-27 2.94175064e-34 2.25417135e-34 7.26677649e-31
  8.86662244e-24 2.64651976e-25 2.19393159e-34 1.10314707e-32
  7.99397751e-33 1.51537324e-31 2.61052728e-29 3.89197827e-32
  4.15371178e-33 2.10219620e-38 4.43000155e-28 2.53988924e-23
  2.02801708e-44 1.19645457e-28 1.46096240e-36 9.33286272e-32
  1.91687326e-37 3.74947507e-47 6.96286888e-45 8.35624400e-17
  9.62642877e-18 5.03565356e-19 3.92972569e-19 2.51722235e-19
  1.90796226e-19 2.17198685e-20 1.54116443e-20 1.10740164e-20
  8.47877782e-21 2.56524073e-21 1.24579552e-21 7.24224985e-22
  5.66257338e-22 2.00828396e-22 1.56610975e-22 6.21282070e-23
  2.83416173e-23 2.19909481e-23 1.38805271e-23 1.06852401e-23
  8.23554505e-24 6.04058106e-24 3.37344807e-24 2.80319039e-25
  2.19035708e-25 1.59488890e-25 7.60643517e-26 4.62995329e-26
  3.60407289e-26 2.75425394e-26 2.14758753e-26 1.22110336e-26
  1.50713099e-27 1.16571448e-27 9.01744870e-28 5.96577311e-28
  1.0760

### Avoiding overflow

To avoid overflow in the calculations of gamma and beta, we can rewrite them as exponentials ($\alpha = e^{l}$) and compute $l$ instead; in this way, all the products turn into sums and the algorithm is robust against overflow even for larger N.

In [60]:
def forward_opt(x, A, b, pi=prob[0]):
    T = len(x)
    alpha = np.zeros((2,T))
    l = np.zeros((2,T))

    alpha[:, 0] = pi * b[:, 0]
    l[0,0] = np.log(alpha[0,0])
    l[1,0] = np.log(alpha[1,0])

    y = np.zeros((2))
    for t in range(1, T):
        for j in range(2):
            y[0] = np.log(b[j,t])+np.log(A[0,j])+l[0,t-1]
            y[1] = np.log(b[j,t])+np.log(A[1,j])+l[1,t-1]
            l[j,t] = np.log(np.exp(y[0])+np.exp(y[1]))
            ymax = y.max()
            l[j,t] = ymax+np.log(np.exp(y[0]-ymax)+np.exp(y[1]-ymax))
            
           # alpha[j,t] = np.exp(l[j,t])
    return l

In [61]:
b=calculate_emission_matrix(B_pars, x)
alphas=forward_algorithm(x, A, b, pi=prob[0])
alphas_opt = np.exp(forward_opt(x,A,b,prob[0]))

In [62]:
print('Alphas calculate directly: \n')
print(alphas[:,40:50], '\n\n')
print('Alphas with log-sum-exp trick: \n')
print(alphas_opt[:,40:50],)

Alphas calculate directly: 

[[5.66257338e-22 2.00828396e-22 1.56610975e-22 6.21282070e-23
  2.83416173e-23 2.19909481e-23 1.38805271e-23 1.06852401e-23
  8.23554505e-24 6.04058106e-24]
 [1.99874233e-45 2.28553375e-40 2.95966870e-46 2.13301422e-51
  3.45850738e-51 2.50125053e-47 7.85709450e-50 2.46437232e-46
  1.77028796e-46 6.64968242e-49]] 


Alphas with log-sum-exp trick: 

[[5.66257338e-22 2.00828396e-22 1.56610975e-22 6.21282070e-23
  2.83416173e-23 2.19909481e-23 1.38805271e-23 1.06852401e-23
  8.23554505e-24 6.04058106e-24]
 [1.99874233e-45 2.28553375e-40 2.95966870e-46 2.13301422e-51
  3.45850738e-51 2.50125053e-47 7.85709450e-50 2.46437232e-46
  1.77028796e-46 6.64968242e-49]]


In [63]:
def backward_opt(x, A, b):
    T = len(x)
    beta = np.zeros((2,T))
    l = np.zeros((2,T))

    beta[:, -1] = 1
    l[0,-1] = np.log(beta[0,-1])
    l[1,-1] = np.log(beta[1,-1])

    y = np.zeros((2))
    for t in range(T-2, -1, -1):
        for j in range(2):
            y[0] = np.log(b[0,t+1])+np.log(A[j,0])+l[0,t+1]
            y[1] = np.log(b[1,t+1])+np.log(A[j,1])+l[1,t+1]
            ymax = y.max()
            l[j,t] = ymax+np.log(np.exp(y[0]-ymax)+np.exp(y[1]-ymax))
            
           # beta[j,t] = np.exp(l[j,t])
    return l

In [64]:
b = calculate_emission_matrix(B_pars, x)
betas = backward_algorithm(x, A, b)
betas_opt = np.exp(backward_opt(x,A,b))

In [65]:
print('betas calculate directly: \n')
print(betas[:,40:50], '\n\n')
print('betas with log-sum-exp trick: \n')
print(betas_opt[:,40:50],)

betas calculate directly: 

[[5.95188680e-25 1.67819872e-24 2.15202004e-24 5.42474946e-24
  1.18916981e-23 1.53258494e-23 2.42807751e-23 3.15416362e-23
  4.09238193e-23 5.57942943e-23]
 [1.21467078e-26 3.42489535e-26 4.39187763e-26 1.10709173e-25
  2.42687717e-25 3.12772436e-25 4.95526022e-25 6.43706862e-25
  8.35179986e-25 1.13865907e-24]] 


betas with log-sum-exp trick: 

[[5.95188680e-25 1.67819872e-24 2.15202004e-24 5.42474946e-24
  1.18916981e-23 1.53258494e-23 2.42807751e-23 3.15416362e-23
  4.09238193e-23 5.57942943e-23]
 [1.21467078e-26 3.42489535e-26 4.39187763e-26 1.10709173e-25
  2.42687717e-25 3.12772436e-25 4.95526022e-25 6.43706862e-25
  8.35179986e-25 1.13865907e-24]]


## Gamma and Psi
\begin{equation}
\gamma_t(i) = \frac{\alpha_t(i)\beta_t(i)}{\sum_{j=1}^{N}\alpha_t(j)\beta_t(j)}
\end{equation}

where $N$ is the number of states in the HMM, $\alpha_t(i)$ is computed at time $t$ and state $i$, and $\beta_t(i)$ is computed at time $t$ and state $i$. This formula computes the probability of being in state $i$ at time $t$, given the observed sequence and the HMM parameters.

The formula for the psi value at time step $t$, current state $i$, and next state $j$ is:

\begin{equation}
\psi_t(i,j) = \frac{\alpha_t(i)a_{ij}b_j(x_{t+1})\beta_{t+1}(j)}{\sum_{i=1}^{N}\sum_{j=1}^{N}\alpha_t(i)a_{ij}b_j(x_{t+1})\beta_{t+1}(j)}
\end{equation}

where $N$ is the number of states in the HMM, $a_{ij}$ is the transition probability from state $i$ to state $j$, $b_j(x_{t+1})$ is the emission probability of the observation $x_{t+1}$ given state $j$, and $\beta_{t+1}(j)$ is the backward variable at time $t+1$ and state $j$. 

In [66]:
#gamma[0][i] = alpha[0][i]*beta[0][i]/(alpha[0][i]*beta[0][i]+alpha[1][i]*beta[1][i] )
#gamma[1][i] =  alpha[1][i]*beta[1][i]/(alpha[0][i]*beta[0][i]+alpha[1][i]*beta[1][i] )
  
def get_gamma(x, alpha, beta):
    n = alpha.shape[0]  # number of states
    T = len(x)
    gamma = np.zeros((n, T))

    for t in range(T):
        norm = np.sum(alpha[:, t] * beta[:, t]) #dot product
        for i in range(n):
            gamma[i, t] = alpha[i, t] * beta[i, t] / norm

    return gamma


def get_psi(x, beta, alpha, A, b):
    n = alpha.shape[0]  # number of states
    psi = np.zeros((n, n, len(x)))
    for t in range(1, len(x)-1):
        for j in range(n):  # z_t = j
            for i in range(n):  # z_t+1 = i
                psi[i][j][t] = calculate_psi(t, i, j, alpha, beta, A, b)
    return psi


def calculate_psi(t, i, j, alpha, beta, A, b):
    denom = alpha[j][t] * beta[j][t] * A[j][i] * b[i][t+1]
    return (alpha[i][t] * A[i][j] * beta[j][t+1] * b[j][t+1]) / denom
    

In [67]:
gamma=get_gamma(x, alphas, betas)
psi=get_psi(x, betas, alphas, A, b)
print(psi)

[[[0.00000000e+00 1.45683948e+00 1.42900452e+00 1.41519458e+00
   2.41513627e+00 2.27223140e+00 1.31529621e+00 4.17068092e+00
   2.95742214e+00 1.68662274e+00 1.34711191e+00 1.34812986e+00
   1.27891433e+00 1.48661039e+00 1.28271835e+00 1.32735034e+00
   3.28340905e+00 2.31052017e+00 3.11712796e+01 1.13138901e+01
   5.56501238e+00 1.28322203e+00 2.36936606e+00 1.29816513e+00
   1.29627063e+01 9.36910302e+03 1.32576101e+00 8.68052337e+00
   1.91165430e+01 1.28142622e+00 1.56113571e+00 1.31932502e+00
   8.78440981e+00 1.40931545e+00 1.39169420e+00 1.30608639e+00
   3.30525620e+00 2.05911860e+00 1.72017750e+00 1.27896795e+00
   2.81960793e+00 1.28233922e+00 2.52077089e+00 2.19211932e+00
   1.28878561e+00 1.58430208e+00 1.29903745e+00 1.29745391e+00
   1.36336968e+00 1.79062518e+00 1.20343166e+01 1.27978694e+00
   1.37336028e+00 2.09676263e+00 1.64287514e+00 1.28464474e+00
   1.30854778e+00 1.28248740e+00 1.75872707e+00 8.10217139e+00
   1.29288176e+00 1.29273204e+00 1.51153062e+00 5.54411

### Viterbi Algorithm

In Viterbi Algorithm we will compute the most likely hidden sequence of the hidden states($z_{1:T}$) that given the visible state($x_{1:T}$).

$$z^{*} = {argmax} \space P(z|x)$$
\
The Viterbi algorithm is an efficient way to make an inference, or prediction, to the hidden states given the model parameters are optimized, and given the observed data. It is best visualized by a trellis to find out how the path is select from a time step to the next. 
\
We will compute $\mu$ for each state which shows us the most likely.\
$$\mu_{z_1} = P(z_1,x_1) = \pi_{i} b_{i}(x) \space \space \space \space \space  for \space the \space initial \space state $$
\
$$\mu_{z_i} = max[P(x_t|z_t)P(z_{t})|z_{t-1} \mu_{t-1}(z_{t-1})]\space \space \space \space \space  for \space other  \space states $$

In [111]:
def viterbi(x, A, b, pi =prob[0]): #maybe need the end state
    """our variables
        x: observed sequence (array of shape (T,))
        A: transition probability matrix (array of shape (2, 2))
        b: emission probability matrix (array of shape (2, T))
        pi:initial state distribution (array of shape (2,))
        output : mu matrix (array of shape (2,T))
    """
    T = len(x)
    T_1 = np.zeros((2,T))
    mu= np.zeros((2,T))
    mu [:,0]= pi*b[:,0]
    T_1[:,0] = pi*b[:,0]

        
    for t in range(1,T):
        for j in range(2):

            T_1[j,t] =np.sum(mu[:,t-1]*A[:,j])*b[j,t]
        for k in range(2):
            mu[k,t] =max(T_1[k,:t])
                   
    return mu
    
    
    

In [127]:
prob = np.array([.5,.5])
A = np.array([[.98,.02],[.02,.98]])
b=calculate_emission_matrix(B_pars, x)
mu= viterbi(x,A,b,pi=prob[0])
print("mu is:", '\n', mu)

mu is: 
 [[3.81169531e-02 3.81169531e-02 3.81169531e-02 3.81169531e-02
  3.81169531e-02 3.81169531e-02 3.81169531e-02 3.81169531e-02
  3.81169531e-02 3.81169531e-02 3.81169531e-02 3.81169531e-02
  3.81169531e-02 3.81169531e-02 3.81169531e-02 3.81169531e-02
  3.81169531e-02 3.81169531e-02 3.81169531e-02 3.81169531e-02
  3.81169531e-02 3.81169531e-02 3.81169531e-02 3.81169531e-02
  3.81169531e-02 3.81169531e-02 3.81169531e-02 3.81169531e-02
  3.81169531e-02 3.81169531e-02 3.81169531e-02 3.81169531e-02
  3.81169531e-02 3.81169531e-02 3.81169531e-02 3.81169531e-02
  3.81169531e-02 3.81169531e-02 3.81169531e-02 3.81169531e-02
  3.81169531e-02 3.81169531e-02 3.81169531e-02 3.81169531e-02
  3.81169531e-02 3.81169531e-02 3.81169531e-02 3.81169531e-02
  3.81169531e-02 3.81169531e-02 3.81169531e-02 3.81169531e-02
  3.81169531e-02 3.81169531e-02 3.81169531e-02 3.81169531e-02
  3.81169531e-02 3.81169531e-02 3.81169531e-02 3.81169531e-02
  3.81169531e-02 3.81169531e-02 3.81169531e-02 3.81169531e-02